In [1]:
import sys
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')
import pickle
import datetime

import pandas as pd
import numpy as np
import time
import pyarrow.parquet as pq
from scipy.sparse import csr_matrix, coo_matrix, vstack, save_npz
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, cv, Pool, sum_models, EShapCalcType, EFeaturesSelectionAlgorithm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

import vaex

import seaborn as sns
import matplotlib.pyplot as plt


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
LOCAL_DATA_PATH = 'context_data'
SPLIT_SEED = 42
DATA_FILE = 'competition_data_final_pqt'
TARGET_FILE = 'public_train.pqt'
SUBMISSION_FILE = 'submit_2.pqt'

In [3]:
id_to_submit = pq.read_table(f'../{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas()
tgt = pq.read_table(f'../{LOCAL_DATA_PATH}/{TARGET_FILE}').to_pandas()

In [4]:
df = vaex.open(f'../{LOCAL_DATA_PATH}/{DATA_FILE}')
#df.head()

In [7]:
%%time

df['date'] = df.date.astype(str)
dates_list = df.date.unique()
dow_dict = {}
dates_dict = {}
for dtx in dates_list:
    dow_dict[dtx] = datetime.datetime.strptime(dtx, '%Y-%m-%d').strftime("%A")
    dates_dict[dtx] = datetime.datetime.strptime(dtx, '%Y-%m-%d')
df['dow'] = df['date'].map(dow_dict)
df['date'] = df['date'].map(dates_dict)

os_dict = {'Apple iOS': 'iOS',
           'iOS': 'iOS',
           'Android': 'Android'}
df['os'] = df.cpe_model_os_type.map(os_dict)

CPU times: total: 5min 25s
Wall time: 53 s


In [19]:
## df[(df.user_id == 167006) & (df.cpe_type_cd == 'plain')].cpe_type_cd = 'smartphone'

In [5]:
def age_bucket(x):
    return bisect.bisect_left([18,25,35,45,55,65], x)

tgt['age_b'] = tgt['age'].map(age_bucket)
tgt

,age,is_male,user_id,age_b
350459,31.0,1,350459,2
188276,35.0,1,188276,2
99002,41.0,0,99002,3
155506,33.0,0,155506,2
213873,54.0,0,213873,4
...,...,...,...,...
225374,49.0,1,225374,4
25776,22.0,1,25776,1
148131,28.0,0,148131,2
205570,28.0,1,205570,2


In [6]:
df_list = []

In [7]:
%%time

agg_pod_req = df.groupby(['user_id', 'part_of_day']).agg({'request_cnt': 'sum'})
agg_dow_req = df.groupby(['user_id', 'dow']).agg({'request_cnt': 'sum'})

df_list.append(agg_pod_req.to_pandas_df()\
    .pivot_table(values='request_cnt', \
                 index='user_id', \
                 columns='part_of_day', \
                 fill_value=0
                )
              )
df_list.append(agg_dow_req.to_pandas_df()\
    .pivot_table(values='request_cnt', \
                 index='user_id', \
                 columns='dow', \
                 fill_value=0
                )
              )

### Requests counts for every part of day and weekday

CPU times: total: 4min 32s
Wall time: 29.8 s


In [8]:
%%time

df_list.append(df.groupby('user_id', agg=vaex.agg.list('price'))\
            .to_pandas_df()\
            .set_index('user_id', drop=True)\
            .apply(lambda x: np.unique(x[0]).mean(), axis=1)
            .to_frame('price')
              )

### Average of unique prices

CPU times: total: 30 s
Wall time: 14.3 s


In [9]:
%%time

df_list.append(df.groupby(['user_id','region_name'])\
            .agg({'region_count': vaex.agg.count(df.request_cnt)})\
            .sort(by=['user_id','region_count'])\
            .to_pandas_df()\
            .groupby('user_id').agg({'region_name': 'last'})
              )

df_list.append(df.groupby(['user_id','city_name'])\
            .agg({'city_count': vaex.agg.count(df.request_cnt)})\
            .sort(by=['user_id','city_count'])\
            .to_pandas_df()\
            .groupby('user_id').agg({'city_name': 'last'})
              )

### Most used region and city

CPU times: total: 9min 54s
Wall time: 1min 8s


In [10]:
%%time

df_list.append(df.groupby('user_id', agg=vaex.agg.list('cpe_manufacturer_name'))\
            .to_pandas_df()\
            .set_index('user_id', drop=True)\
            .apply(lambda x: x[0][0], axis=1)\
            .to_frame('company')
              )

### cpe_manufacturer_name

CPU times: total: 59.2 s
Wall time: 30.1 s


In [11]:
%%time

df_list.append(df.groupby('user_id', agg=vaex.agg.list('cpe_model_name'))\
            .to_pandas_df()\
            .set_index('user_id', drop=True)\
            .apply(lambda x: x[0][0], axis=1)\
            .to_frame('model')
              )

### cpe_model_name

CPU times: total: 1min 14s
Wall time: 43.6 s


In [12]:
%%time

df_list.append(df.groupby('user_id', agg=vaex.agg.list('os'))\
            .to_pandas_df()\
            .set_index('user_id', drop=True)\
            .apply(lambda x: x[0][0], axis=1)\
            .to_frame('os')
              )

### OS

CPU times: total: 1min 58s
Wall time: 1min 22s


In [22]:
with open('feat_gen_df_list.pickle', 'wb') as outp:
    pickle.dump(df_list, outp, pickle.HIGHEST_PROTOCOL)

In [42]:
%%time

df_list.append(df.groupby('user_id')\
                    .agg({'region_name_count': vaex.agg.nunique(df.region_name),
                          'city_name_count': vaex.agg.nunique(df.city_name),
                          'req_max': vaex.agg.max(df.request_cnt),
                          'req_sum': vaex.agg.sum(df.request_cnt),
                          'id_rows': vaex.agg.count(df.cpe_manufacturer_name),   ### cpe_manufacturer_name только 1
                          'days': vaex.agg.nunique(df.date)})\
                    .to_pandas_df()\
                    .set_index('user_id', drop=True)
              )

CPU times: total: 3min 59s
Wall time: 1min 22s


In [53]:
df_list[0]['req_sum'] = df_list[-1].req_sum
for col in df_list[0].columns[:-1]:
    df_list[0][col] = df_list[0][col] / df_list[0]['req_sum']
df_list[0] = df_list[0][df_list[0].columns[:-1]]

df_list[1]['req_sum'] = df_list[-1].req_sum
for col in df_list[1].columns[:-1]:
    df_list[1][col] = df_list[1][col] / df_list[1]['req_sum']
df_list[1] = df_list[1][df_list[1].columns[:-1]]

###  Requests counts --> normalize for parts of day and for weekdays

In [32]:
with open('feat_gen_df_list.pickle', 'wb') as outp:
    pickle.dump(df_list, outp, pickle.HIGHEST_PROTOCOL)

In [60]:
feat_df = pd.concat(df_list, axis=1)
feat_df

,day,evening,morning,night,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,...,city_name,company,model,os,region_name_count,city_name_count,req_max,req_sum,id_rows,days
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.554404,0.321244,0.119171,0.005181,0.056995,0.020725,0.134715,0.108808,0.036269,0.440415,...,Москва,Samsung,Galaxy J1 2016 LTE Dual,Android,1,1,5,193,131,17
1,0.346705,0.295129,0.322827,0.035339,0.127985,0.209169,0.102197,0.098376,0.122254,0.150907,...,Москва,Xiaomi,Mi 9,Android,3,6,6,1047,700,19
2,0.481752,0.316302,0.187348,0.014599,0.153285,0.128954,0.148418,0.150852,0.104623,0.128954,...,Печора,Huawei,Honor 9 Lite,Android,1,1,4,411,356,50
3,0.352727,0.454545,0.178182,0.014545,0.240000,0.185455,0.065455,0.116364,0.123636,0.090909,...,Воронеж,Huawei Device Company Limited,P Smart 2021,Android,1,1,5,275,188,15
4,0.348777,0.265122,0.371943,0.014157,0.212355,0.164736,0.185328,0.141570,0.118404,0.072072,...,Анапа,Huawei,Nova 3,Android,5,9,5,777,591,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415312,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,Моршанск,Huawei,Honor 7S Dual,Android,1,1,1,1,1,1
415313,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,Саратов,Xiaomi,Redmi 6A,Android,1,1,1,1,1,1
415314,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,Новосибирск,Huawei Device Company Limited,Honor 10X Lite,Android,1,1,1,1,1,1


In [61]:
feat_df.to_csv('feat_gen_df2.csv')

In [47]:
######________#######

In [15]:
feat_df = pd.read_csv('feat_gen_df2.csv', index_col='user_id')

In [5]:
df['date'] = df.date.astype(str)
dates_list = df.date.unique()
dates_dict = {}
for dtx in dates_list:
    dates_dict[dtx] = datetime.datetime.strptime(dtx, '%Y-%m-%d')
df['date'] = df['date'].map(dates_dict)

In [7]:
%%time

df_dates_range = df[['user_id', 'date']]\
                    .groupby('user_id')\
                    .agg({'date': 'list'})\
                    .to_pandas_df()\
                    .apply(lambda x: ((max(x['date']) - min(x['date'])) / 86400000000).astype(int), axis=1)

CPU times: total: 1min 32s
Wall time: 37 s


In [16]:
feat_df = pd.concat([feat_df, df_dates_range.rename('dates_range')], axis=1).rename_axis('user_id')
feat_df.to_csv('feat_gen_df3.csv')
feat_df

,day,evening,morning,night,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,...,company,model,os,region_name_count,city_name_count,req_max,req_sum,id_rows,days,dates_range
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.554404,0.321244,0.119171,0.005181,0.056995,0.020725,0.134715,0.108808,0.036269,0.440415,...,Samsung,Galaxy J1 2016 LTE Dual,Android,1,1,5,193,131,17,18
1,0.346705,0.295129,0.322827,0.035339,0.127985,0.209169,0.102197,0.098376,0.122254,0.150907,...,Xiaomi,Mi 9,Android,3,6,6,1047,700,19,20
2,0.481752,0.316302,0.187348,0.014599,0.153285,0.128954,0.148418,0.150852,0.104623,0.128954,...,Huawei,Honor 9 Lite,Android,1,1,4,411,356,50,57
3,0.352727,0.454545,0.178182,0.014545,0.240000,0.185455,0.065455,0.116364,0.123636,0.090909,...,Huawei Device Company Limited,P Smart 2021,Android,1,1,5,275,188,15,16
4,0.348777,0.265122,0.371943,0.014157,0.212355,0.164736,0.185328,0.141570,0.118404,0.072072,...,Huawei,Nova 3,Android,5,9,5,777,591,20,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415312,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,Huawei,Honor 7S Dual,Android,1,1,1,1,1,1,0
415313,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,Xiaomi,Redmi 6A,Android,1,1,1,1,1,1,0
415314,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,Huawei Device Company Limited,Honor 10X Lite,Android,1,1,1,1,1,1,0


In [20]:
### Reduction of URLs

In [24]:
turbo_dict = {url: url for url in df['url_host'].unique()}
turbo_list = [url for url in df['url_host'].unique() if url.find('.turbopages') > -1]
for url in turbo_list:
    if url.startswith('xn'):
        value = url.split('.turbopages')[0]\
                   .encode('utf-8').decode('idna')\
                   .replace('--', '!').replace('-', '.').replace('!', '-')\
                   .encode('idna').decode('utf-8')
    else:
        value = url.split('.turbopages')[0]\
                   .replace('--', '!').replace('-', '.').replace('!', '-')
    turbo_dict[url] = value
    
df['url_host'] = df['url_host'].map(turbo_dict)

In [25]:
ampproject_dict = {url: url for url in df['url_host'].unique()}
ampproject_list = [url for url in df['url_host'].unique() if url.find('.ampproject') > -1]
for url in ampproject_list:
    if url.startswith('0-'):
        value = url.split('.cdn.ampproject.org')[0][2:-2]\
                   .replace('--', '!').replace('-', '.').replace('!', '-')
    else:
        value = url.split('.cdn.ampproject.org')[0]\
                   .replace('--', '!').replace('-', '.').replace('!', '-')
    ampproject_dict[url] = value
    
df['url_host'] = df['url_host'].map(ampproject_dict)

In [26]:
amp_dict = {url: url for url in df['url_host'].unique()}
amp_list = [url for url in df['url_host'].unique() if url.startswith('amp.')]
for url in amp_list:
    value = url.split('amp.')[1]
    amp_dict[url] = value
    
df['url_host'] = df['url_host'].map(amp_dict)

In [27]:
investing_dict = {url: url for url in df['url_host'].unique()}
investing_list = [url for url in df['url_host'].unique() if url.find('investing.com') > -1]
for url in investing_list:
    value = 'investing.com'
    investing_dict[url] = value
    
df['url_host'] = df['url_host'].map(investing_dict)

In [28]:
adriver_dict = {url: url for url in df['url_host'].unique()}
adriver_list = [url for url in df['url_host'].unique() if url.find('adriver') > -1]
for url in adriver_list:
    value = 'adriver.ru'
    adriver_dict[url] = value
    
df['url_host'] = df['url_host'].map(adriver_dict)

In [29]:
bg_dict = {url: url for url in df['url_host'].unique()}
bg_dict['rt.bongacams21.com'] = 'rt.bongacams14.com'
df['url_host'] = df['url_host'].map(bg_dict)

In [30]:
m_dict = {url: url for url in df['url_host'].unique()}
m_list = [url for url in df['url_host'].unique() if url.startswith('m.') \
          and url not in ['m.vk.com', 'm.ok.ru', 'm.zen.yandex.ru']]
for url in m_list:
    value = url[2:]
    m_dict[url] = value
    
df['url_host'] = df['url_host'].map(m_dict)

In [31]:
%%time
data_agg = df[['user_id', 'url_host', 'request_cnt']].\
    groupby(['user_id', 'url_host']).agg([('request_cnt', "sum")])

url_set = set(data_agg['url_host'].unique())
print(f'{len(url_set)} urls')
url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}
usr_set = set(data_agg['user_id'].unique())
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

190387 urls
415317 users
CPU times: total: 22min 34s
Wall time: 2min 2s


In [33]:
%%time
values = data_agg['request_cnt'].values.astype(np.uint16)
rows = data_agg['user_id'].map(usr_dict).to_numpy()
cols = data_agg['url_host'].map(url_dict).to_numpy()
mat = csr_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
save_npz('mat_ed.npz', mat)
mat

CPU times: total: 32.9 s
Wall time: 15.9 s


<415317x190387 sparse matrix of type '<class 'numpy.uint16'>'
	with 31305376 stored elements in Compressed Sparse Row format>

In [37]:
mat.nonzero()[0].shape

(31305376,)

In [39]:
mat.data.shape

(31305376,)

In [40]:
with open('url_usr_dicts_ed.pickle', 'wb') as outp:
    pickle.dump((url_dict, usr_dict), outp, pickle.HIGHEST_PROTOCOL)